# Detección de plagio

Instalación de librerias necesarias para el tratamiento deltexto.




In [1]:
#librerias necesarias
import nltk
import re
from nltk import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
#texto que se desea analizar
text = input('Ingrese el texto que desea estudiar:')
text

# Ejemplo = "Una variable aleatoria representa un valor numérico influenciado por el azar. No se pueden predecir con certeza sus resultados individuales, sino solo su distribución probabilística. Por ejemplo, en una epidemia de cólera, aunque se sabe que algunas personas se enfermarán y otras no, no se puede determinar con certeza el resultado para cada persona. La probabilidad de enfermarse solo puede estimarse. Para trabajar eficazmente con variables aleatorias, es necesario realizar muchos experimentos y cuantificar sus resultados para que se les puedan asignar valores numéricos. Esto crea una relación funcional entre los elementos del espacio muestral asociado con el experimento y los números reales."

Ingrese el texto que desea estudiar:Una variable aleatoria representa un valor numérico influenciado por el azar. No se pueden predecir con certeza sus resultados individuales, sino solo su distribución probabilística. Por ejemplo, en una epidemia de cólera, aunque se sabe que algunas personas se enfermarán y otras no, no se puede determinar con certeza el resultado para cada persona. La probabilidad de enfermarse solo puede estimarse. Para trabajar eficazmente con variables aleatorias, es necesario realizar muchos experimentos y cuantificar sus resultados para que se les puedan asignar valores numéricos. Esto crea una relación funcional entre los elementos del espacio muestral asociado con el experimento y los números reales.


'Una variable aleatoria representa un valor numérico influenciado por el azar. No se pueden predecir con certeza sus resultados individuales, sino solo su distribución probabilística. Por ejemplo, en una epidemia de cólera, aunque se sabe que algunas personas se enfermarán y otras no, no se puede determinar con certeza el resultado para cada persona. La probabilidad de enfermarse solo puede estimarse. Para trabajar eficazmente con variables aleatorias, es necesario realizar muchos experimentos y cuantificar sus resultados para que se les puedan asignar valores numéricos. Esto crea una relación funcional entre los elementos del espacio muestral asociado con el experimento y los números reales.'

Luego, de que el texto sea ingresado, este se limpia

In [3]:
# Quita saltos de línea
text = re.sub(r'\n',' ',text)
# Quita asteriscos
text = re.sub(r'\*',' ',text)
# Quita Espacios extra
text = re.sub(r'\s\s+',' ',text)
# Quita espacios iniciales y finales
text = text.strip()
# Convierte todos los caracteres en mayúscula a minúscula
text = text.lower()
text

'una variable aleatoria representa un valor numérico influenciado por el azar. no se pueden predecir con certeza sus resultados individuales, sino solo su distribución probabilística. por ejemplo, en una epidemia de cólera, aunque se sabe que algunas personas se enfermarán y otras no, no se puede determinar con certeza el resultado para cada persona. la probabilidad de enfermarse solo puede estimarse. para trabajar eficazmente con variables aleatorias, es necesario realizar muchos experimentos y cuantificar sus resultados para que se les puedan asignar valores numéricos. esto crea una relación funcional entre los elementos del espacio muestral asociado con el experimento y los números reales.'

Se tokeniza el texto en frases.

In [4]:
sentences = sent_tokenize(text)

print('Frases:\n')
for i,sentence in enumerate(sentences):
    print(f"Frase {i}:\n{sentence}\n")

Frases:

Frase 0:
una variable aleatoria representa un valor numérico influenciado por el azar.

Frase 1:
no se pueden predecir con certeza sus resultados individuales, sino solo su distribución probabilística.

Frase 2:
por ejemplo, en una epidemia de cólera, aunque se sabe que algunas personas se enfermarán y otras no, no se puede determinar con certeza el resultado para cada persona.

Frase 3:
la probabilidad de enfermarse solo puede estimarse.

Frase 4:
para trabajar eficazmente con variables aleatorias, es necesario realizar muchos experimentos y cuantificar sus resultados para que se les puedan asignar valores numéricos.

Frase 5:
esto crea una relación funcional entre los elementos del espacio muestral asociado con el experimento y los números reales.



Se realiza una busqueda en Google Search de cada una de las frases, se guardan los links de los primeros 5 resultados de cada busqueda y se seleccionan los 10 links mas frequentes de todas las busquedas.

In [5]:
from googlesearch import search
from nltk import FreqDist

results = []
for sentence in sentences:
  # obtiene el url de los primeros 5 resultados de una frase al buscarlo en el navegador de Google
  for url in search(sentence, lang = 'es', num = 5, start = 0, stop = 5, pause = 2):
    results.append(url)

# Selecciona las 10 urls con mayor frecuencia
freq_urls = [w for (w,i) in FreqDist(results).most_common(10)]
freq_urls

['https://es.wikipedia.org/wiki/Variable_aleatoria',
 'https://es.wikipedia.org/wiki/Variable_estad%C3%ADstica',
 'http://www.ulpgc.es/hege/almacen/download/27/27373/temavariablesaleatorias.pdf',
 'https://www.ugr.es/~eues/webgrupo/Docencia/MonteroAlonso/estadisticaII/tema2.pdf',
 'http://bibliotecadigital.econ.uba.ar/download/libros/Landro-Gonzalez_Teoria-general-de-las-variables-aleatorias-2018.pdf',
 'https://es.wikipedia.org/wiki/Probabilidad',
 'https://wpd.ugr.es/~bioestad/bioestadistica/tema-4/',
 'https://ienci.if.ufrgs.br/index.php/ienci/article/download/3028/826',
 'http://home.ubalt.edu/ntsbarsh/opre640s/spanishp.htm',
 'https://mises.org/es/library/incertidumbre']

Se installa y cargan las librerias "html2text", "requests" "BeautifulSoup" y "time", que ayudan a extraer información de paginas web.

In [6]:
pip install html2text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import html2text
import requests
from bs4 import BeautifulSoup
import time

Se extrae el texto de las paginas de la busqueda (si se puede).

In [8]:
# Rompe defensa dummy anti-robot
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
results_texts = []
results_unopened = []


for url in freq_urls:
  try:
    # interrumpe la ejecución del programa durante 0,01 segundos
    time.sleep(0.01)
    # Obtiene conexión con una url
    res = requests.get(url,headers=headers)
    # Extrae contenido de la url
    html_page = res.content
    # Parsing del contenido
    soup = BeautifulSoup(html_page, 'html.parser')
    result_text = soup.find_all('p')
    # Convierte html a texto
    clean_text = html2text.html2text(''.join(str(result_text)))
    # Limpia el texto
    clean_text = re.sub(r'\([^)]*\)',' ',clean_text)
    clean_text = re.sub(r'[\[\]\(\)]','',clean_text)
    clean_text = re.sub(r'\n',' ',clean_text)
    clean_text = re.sub(r'\*',' ',clean_text)
    clean_text = re.sub(r"[\\']",'"',clean_text)
    clean_text = re.sub(r'([^.]*$)','',clean_text)
    clean_text = re.sub('\s\s+', ' ',clean_text)
    clean_text = re.sub('\.\s,\s', '. ',clean_text)
    clean_text = clean_text.strip()
    clean_text = clean_text.lower()
    if clean_text:
      results_texts.append((url,clean_text))
    elif not re.search('403',str(res)):
      results_unopened.append(url)
  except:
    # Si surge un error o no se puede extraer contenido de una pagina
    results_unopened.append(url)

len(results_texts)

6

Se instala el modelo que compara similaridad semantica en español.

In [ ]:
pip install -U sentence-transformers

In [10]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

In [11]:
posible_similarities = []
for r in results_texts:
  sent_result_text = sent_tokenize(r[1])

  for sentence in sentences:
    total_sent = [sentence]
    total_sent.extend(sent_result_text)

    # Crea una lista de tripletas (score, id1, id2), id1 y id2 son los indices de de las frases comparadas
    # Por cada frase devuelve solo la mas semanticamente similar a esta
    paraphrases = util.paraphrase_mining(model, total_sent, top_k=1)

    for paraphrase in paraphrases:
      score, i, j = paraphrase

      # Selecciona las tripletas, en donde esta la frase con la similaridad mas alta a una frase del texto ingresado por cada pagina.
      if total_sent[i] == sentence:

        # Selecciona las tripletas con el puntaje de similaridad mayor o igual a 0.96, el cual indica alta probabilidad de plagio, y las imprime.
        if score >= 0.96:
          print(f"El siguente pasaje muy probablemente proviene de {r[0]}:")
          print("'{}' tiene un porcentaje de similaridad de:".format(sentence))
          print("{:.2f}% \t con '{}' ".format(score*100, total_sent[j]))
        else:

          # Selecciona las tripletas con el puntaje de similaridad entre 0.7 y 0.85, y las guarda
          # Este puntaje aunque no indica plagio, indica que en las paginas hay contenido relacionado
          if score > 0.7 and score < 0.85:
            posible_similarities.append(r[0])

          # Selecciona las tripletas con el puntaje de similaridad entre 0.85 y 0.96, el cual indica una posibilidad de plagio, y las imprime.
          elif score >= 0.85 and score < 0.96:
            print("'{}' tiene un porcentaje de similaridad de:".format(sentence))
            print("{:.2f}% \t con '{}' ".format(score*100, total_sent[j]))
            print(f"De: {r[0]}")

# Imprime los links de las paginas en donde puede haber contenido altamente relacionado con el texto ingresado, que puede ser de interes
if len(posible_similarities) > 0:
  print("Pueden haber entradas relacionadas al texto ingresado en:")
for p in posible_similarities:
  print(p)

El siguente pasaje muy probablemente proviene de https://es.wikipedia.org/wiki/Variable_aleatoria:
'una variable aleatoria representa un valor numérico influenciado por el azar.' tiene un porcentaje de similaridad de:
97.92% 	 con 'una variable aleatoria puede concebirse como un valor numérico que está afectado por el azar.' 
'no se pueden predecir con certeza sus resultados individuales, sino solo su distribución probabilística.' tiene un porcentaje de similaridad de:
86.07% 	 con 'dada una variable aleatoria no es posible conocer con certeza el valor que tomará esta al ser medida o determinada, aunque sí se conoce que existe una distribución de probabilidad asociada al conjunto de valores posibles.' 
De: https://es.wikipedia.org/wiki/Variable_aleatoria
'por ejemplo, en una epidemia de cólera, aunque se sabe que algunas personas se enfermarán y otras no, no se puede determinar con certeza el resultado para cada persona.' tiene un porcentaje de similaridad de:
91.99% 	 con 'por ejemplo